In [1]:
import numpy as np
import matplotlib.pyplot as plt
import dolfinx
from dolfinx import fem, mesh, plot
import ufl
from mpi4py import MPI
import gmsh
import pyvista as pv

# Define mesh parameters
gmsh.initialize()
gmsh.model.add("Silicon_Aluminum_UChannel")

# Define U-channel geometry (Aluminum)
lc = 0.01  # Mesh size
gmsh.model.occ.addRectangle(-0.05, -0.05, 0, 0.1, 0.01)  # Bottom
gmsh.model.occ.addRectangle(-0.05, -0.05, 0, 0.01, 0.1)  # Left wall
gmsh.model.occ.addRectangle(0.04, -0.05, 0, 0.01, 0.1)  # Right wall

# Define Silicon Block
gmsh.model.occ.addRectangle(-0.02, -0.04, 0, 0.04, 0.08)  # Inside block

# Synchronize geometry
gmsh.model.occ.synchronize()

# Mesh the geometry
gmsh.model.mesh.generate(2)
gmsh.write("mesh.msh")
gmsh.finalize()

# Load mesh into FEniCS
from dolfinx.io import gmshio
from mpi4py import MPI

# Load mesh with correct geometric dimension
domain, cell_tags, facet_tags = gmshio.read_from_msh("mesh.msh", MPI.COMM_WORLD, 0, gdim=2)

print(f"Mesh Loaded: {domain.topology.dim}D with {domain.topology.index_map(0).size_local} nodes and {domain.topology.index_map(domain.topology.dim).size_local} elements")


# Define function space for elasticity problem
V = fem.FunctionSpace(domain, ufl.VectorElement("Lagrange", domain.ufl_cell(), 2))

# Define material properties (Aluminum & Silicon)
E_al = 70e9  # Aluminum Young's modulus (Pa)
nu_al = 0.33  # Aluminum Poisson's ratio

E_si = 160e9  # Silicon Young's modulus (Pa)
nu_si = 0.22  # Silicon Poisson's ratio

# Define elasticity tensors
def elasticity(E, nu):
    mu = E / (2 * (1 + nu))
    lmbda = E * nu / ((1 + nu) * (1 - 2 * nu))
    return 2 * mu * ufl.sym(ufl.grad(ufl.TrialFunction(V))) + lmbda * ufl.div(ufl.TrialFunction(V)) * ufl.Identity(len(V))

# Define boundary conditions
def left_wall(x): return np.isclose(x[0], -0.05)
def bottom(x): return np.isclose(x[1], -0.05)
def right_wall(x): return np.isclose(x[0], 0.05)

bcs = [fem.dirichletbc(fem.Constant(domain, np.array([0.0, 0.0], dtype=np.float64)), fem.locate_dofs_geometrical(V, bottom))]

# Define weak form for elasticity
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
a = ufl.inner(elasticity(E_al, nu_al), ufl.grad(v)) * ufl.dx
L = ufl.inner(fem.Constant(domain, np.array([0, -1000], dtype=np.float64)), v) * ufl.dx  # Gravity Load

# Solve elasticity problem
problem = fem.petsc.LinearProblem(a, L, bcs=bcs)
u_sol = problem.solve()

# Extract displacement values
u_values = u_sol.x.array.reshape((-1, 2))

# Convert mesh to PyVista for visualization
topology, cell_types, geometry = plot.create_vtk_mesh(domain, domain.topology.dim)
grid = pv.UnstructuredGrid(topology, cell_types, geometry)
grid["Displacement"] = np.linalg.norm(u_values, axis=1)

# Plot result
plotter = pv.Plotter()
plotter.add_mesh(grid, scalars="Displacement", cmap="coolwarm", show_edges=True)
plotter.show()


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 20%] Meshing curve 4 (Line)
Info    : [ 30%] Meshing curve 5 (Line)
Info    : [ 40%] Meshing curve 6 (Line)
Info    : [ 40%] Meshing curve 7 (Line)
Info    : [ 50%] Meshing curve 8 (Line)
Info    : [ 60%] Meshing curve 9 (Line)
Info    : [ 60%] Meshing curve 10 (Line)
Info    : [ 70%] Meshing curve 11 (Line)
Info    : [ 70%] Meshing curve 12 (Line)
Info    : [ 80%] Meshing curve 13 (Line)
Info    : [ 90%] Meshing curve 14 (Line)
Info    : [ 90%] Meshing curve 15 (Line)
Info    : [100%] Meshing curve 16 (Line)
Info    : Done meshing 1D (Wall 0.000617098s, CPU 0.000898s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 30%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 60%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : [ 80%] Meshing surface 4 (Plane, Frontal-Delaunay)


IndexError: index -1 is out of bounds for axis 0 with size 0

In [ ]:
print(dir(mesh_data))


In [ ]:
from dolfinx.io import gmshio
from mpi4py import MPI

# Load mesh data
mesh_data = gmshio.read_from_msh("mesh.msh", MPI.COMM_WORLD, 0, gdim=2)

# Extract the actual mesh from `mesh_data`
domain = mesh_data.mesh

# Check if `mesh_data` contains tagging information
if hasattr(mesh_data, "cell_markers"):
    cell_tags = mesh_data.cell_markers
else:
    cell_tags = None

if hasattr(mesh_data, "facet_markers"):
    facet_tags = mesh_data.facet_markers
else:
    facet_tags = None

# Print mesh details
print(f"Mesh Loaded: {domain.topology.dim}D with {domain.topology.index_map(0).size_local} nodes and {domain.topology.index_map(domain.topology.dim).size_local} elements")

# Print extracted tags
print(f"Cell Tags: {cell_tags}")
print(f"Facet Tags: {facet_tags}")
print(domain)


In [ ]:
print(domain)
print(domain.geometry.x.shape)  # Check node coordinates
print(domain.topology.dim)  # Check mesh dimension


In [ ]:
import dolfinx
import pyvista as pv
import numpy as np

# For headless environments: Use off-screen rendering
pv.start_xvfb()  # Requires Xvfb installation (apt-get install xvfb)

# Extract mesh data
topology, cell_types, geometry = dolfinx.plot.vtk_mesh(domain, domain.topology.dim)

# Create PyVista grid
grid = pv.UnstructuredGrid(topology, cell_types, geometry)

# Plot without GUI display
plotter = pv.Plotter(off_screen=True)  # <-- Critical change
plotter.add_mesh(grid, show_edges=True, color="white")
plotter.view_xy()

# Save to file instead of interactive display
plotter.screenshot("mesh.png")

In [ ]:
import dolfinx
import pyvista as pv
import numpy as np

# Extract PyVista-compatible data using the updated function name
topology, cell_types, geometry = dolfinx.plot.vtk_mesh(domain, domain.topology.dim)

# Create a PyVista UnstructuredGrid
grid = pv.UnstructuredGrid(topology, cell_types, geometry)

# Plot the mesh
plotter = pv.Plotter()
plotter.add_mesh(grid, show_edges=True, color="white")
plotter.view_xy()
plotter.show()
